In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

/home/arman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [35]:
N = 10000
X = np.random.normal(size=(N, 4)).astype(np.float32)
y = (X[:, 0] < X[:, 1]) & (X[:, 1] < X[:, 2]) & (X[:, 2] < X[:, 3])
y = np.array([int(x) for x in y]).astype(np.float32)

In [36]:
TRAIN_SIZE = int(N*0.8)
train_x, test_x = X[:TRAIN_SIZE], X[TRAIN_SIZE:]
train_y, test_y = y[:TRAIN_SIZE], y[TRAIN_SIZE:]

In [39]:
def train_input_fn(features, labels, batch_size=128):
    # Can do all features as one:
    # dataset = tf.data.Dataset.from_tensor_slices(({'feats': features}, labels))
    # Or separate "columns":
    dataset = tf.data.Dataset.from_tensor_slices(({'feat0': features[:,0],
                                                   'feat1': features[:, 1],
                                                   'feat2': features[:, 2],
                                                   'feat3': features[:, 2]},
                                                   labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset

In [40]:
def eval_input_fn(features, labels, batch_size=32):
    feats = {'feat0': features[:,0], 'feat1': features[:, 1],
             'feat2': features[:, 2],'feat3': features[:, 3]}
    
    if labels is None:
        inputs = feats
    else:
        inputs = (feats, labels)

    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    dataset = dataset.batch(batch_size)

    return dataset

In [41]:
my_feature_columns = []
for key in ['feat0', 'feat1', 'feat2', 'feat3']:
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [49]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[128, 128, 64, 64],
    n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpi_8eao1a', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f57246fdeb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [50]:
classifier.train(input_fn=lambda: train_input_fn(train_x, train_y, batch_size=100), steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpi_8eao1a/model.ckpt.
INFO:tensorflow:loss = 62.778046, step = 0
INFO:tensorflow:global_step/sec: 531.846
INFO:tensorflow:loss = 14.604319, step = 100 (0.189 sec)
INFO:tensorflow:global_step/sec: 656.906
INFO:tensorflow:loss = 9.106498, step = 200 (0.152 sec)
INFO:tensorflow:global_step/sec: 754.17
INFO:tensorflow:loss = 12.889675, step = 300 (0.133 sec)
INFO:tensorflow:global_step/sec: 788.557
INFO:tensorflow:loss = 6.4205136, step = 400 (0.127 sec)
INFO:tensorflow:global_step/sec: 758.812
INFO:tensorflow:loss = 7.17578, step = 500 (0.132 sec)
INFO:tensorflow:global_step/sec: 750.716
INFO:tensorflow:loss = 9.299761, step = 600 (0.133 sec)
INFO:tensorflow:global_step/sec: 817.124
INFO:tensorflow:loss 

INFO:tensorflow:global_step/sec: 669.765
INFO:tensorflow:loss = 6.779358, step = 8100 (0.148 sec)
INFO:tensorflow:global_step/sec: 701.68
INFO:tensorflow:loss = 4.941165, step = 8200 (0.143 sec)
INFO:tensorflow:global_step/sec: 627.379
INFO:tensorflow:loss = 8.624987, step = 8300 (0.160 sec)
INFO:tensorflow:global_step/sec: 708.518
INFO:tensorflow:loss = 4.0336914, step = 8400 (0.141 sec)
INFO:tensorflow:global_step/sec: 718.859
INFO:tensorflow:loss = 10.393011, step = 8500 (0.138 sec)
INFO:tensorflow:global_step/sec: 755.552
INFO:tensorflow:loss = 6.646118, step = 8600 (0.133 sec)
INFO:tensorflow:global_step/sec: 542.19
INFO:tensorflow:loss = 7.6087093, step = 8700 (0.183 sec)
INFO:tensorflow:global_step/sec: 698.913
INFO:tensorflow:loss = 9.296492, step = 8800 (0.143 sec)
INFO:tensorflow:global_step/sec: 715.379
INFO:tensorflow:loss = 6.709508, step = 8900 (0.140 sec)
INFO:tensorflow:global_step/sec: 726.157
INFO:tensorflow:loss = 3.4169483, step = 9000 (0.139 sec)
INFO:tensorflow:gl

In [51]:
eval_result = classifier.evaluate(input_fn=lambda: eval_input_fn(test_x, test_y, batch_size=100))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-03:40:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpi_8eao1a/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-03:40:15
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.945, accuracy_baseline = 0.9585, auc = 0.88133436, auc_precision_recall = 0.2563562, average_loss = 0.20770557, global_step = 10000, label/mean = 0.0415, loss = 20.770557, precision = 0.18604651, prediction/mean = 0.037226032, recall = 0.09638554

Test set accuracy: 0.945



In [52]:
print("full eval results")
print(eval_result)

full eval results
{'accuracy': 0.945, 'accuracy_baseline': 0.9585, 'auc': 0.88133436, 'auc_precision_recall': 0.2563562, 'average_loss': 0.20770557, 'label/mean': 0.0415, 'loss': 20.770557, 'precision': 0.18604651, 'prediction/mean': 0.037226032, 'recall': 0.09638554, 'global_step': 10000}
